In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


>> User:Good evening


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


DialoGPT: Good morning
>> User:How are you?
DialoGPT: I'm good, how are you?
>> User:Me too
DialoGPT: I'm good
>> User:That's good
DialoGPT: It is.
>> User:bey
DialoGPT: I'm good


In [3]:
import pandas as pd

In [2]:
dados_pedidos = {
    "numero_pedido": ["12345", "67890", "11121", "22232"],
    "status": ["Shipped", "Processing", "Delivered", "Cancelled"]
}
df_status_pedidos = pd.DataFrame(dados_pedidos)

NameError: name 'pd' is not defined

In [5]:
df_status_pedidos

,numero_pedido,status
0,12345,Shipped
1,67890,Processing
2,11121,Delivered
3,22232,Cancelled


In [6]:
def verifica_status_pedido(numero_pedido):
    try:
        status = df_status_pedidos[df_status_pedidos['numero_pedido'] == numero_pedido]['status'].iloc[0]

        return f'The status of your order {numero_pedido} is: {status}'
    except:
        return f'Orrder number not found. please check and try again'

In [9]:
palavras_chave_status = ['order', 'order status', 'status of my order', 'check my order', 'track my order', 'order update']

In [16]:
ids_historico_chat = None

while True:
  input_usuario = input('You: ')

  if input_usuario.lower() in ['quite', 'exit', 'stop']:
      print('Bot: Goodbey!')
      break

  if any(keyword in input_usuario.lower() for keyword in palavras_chave_status):
    numero_pedido = input('Could you please enter your order number? ')
    resposta = verifica_status_pedido(numero_pedido)

  else:
    novo_usuario_input_ids = tokenizer.encode(input_usuario + tokenizer.eos_token, return_tensors='pt')

    if ids_historico_chat is not None:
      bot_input_ids = torch.cat([ids_historico_chat, novo_usuario_input_ids], dim=-1)

    else:
      bot_input_ids = novo_usuario_input_ids

    ids_historico_chat = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id
    )
    resposta = tokenizer.decode(ids_historico_chat[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

  print(f'Bot: {resposta}')

You: i would like to check the status of my order
Could you please enter your order number? 12345
Bot: The status of your order 12345 is: Shipped
You: stop
Bot: Goodbey!


In [17]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [5]:
# Importações necessárias
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import gradio as gr
import pandas as pd

# Carregando o tokenizer e o modelo
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Criando dataframe de status dos pedidos
dados_pedidos = {
    "numero_pedido": ["12345", "67890", "11121", "22232"],
    "status": ["Shipped", "Processing", "Delivered", "Cancelled"]
}
df_status_pedidos = pd.DataFrame(dados_pedidos)

# Função para verificar status do pedido
def verifica_status_pedido(numero_pedido):
    try:
        status = df_status_pedidos[df_status_pedidos['numero_pedido'] == numero_pedido]['status'].iloc[0]

        return f'The status of your order {numero_pedido} is: {status}'
    except:
        return f'Orrder number not found. please check and try again'

# Lista de palavras-chaves para o status
palavras_chave_status = ['order', 'order status', 'status of my order', 'check my order', 'track my order', 'order update']

def responder(input_usuario, ids_historico_chat=None):
  if any(keyword in input_usuario.lower() for keyword in palavras_chave_status):
    return 'Could you please enter your order number? ', ids_historico_chat

  else:
    novo_usuario_input_ids = tokenizer.encode(input_usuario + tokenizer.eos_token, return_tensors='pt')

    if ids_historico_chat is not None:
      bot_input_ids = torch.cat([ids_historico_chat, novo_usuario_input_ids], dim=-1)

    else:
      bot_input_ids = novo_usuario_input_ids

    ids_historico_chat = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id
    )
    resposta = tokenizer.decode(ids_historico_chat[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

  return resposta, ids_historico_chat

# Criando interface
with gr.Blocks() as app:
  chatbot = gr.Chatbot()
  msg = gr.Textbox(placeholder='Type your message here...')

  estado = gr.State(None)
  aguardando_numero_pedido = gr.State(False)

  def processa_entrada(input_usuario, historico, ids_historico_chat, aguardando_numero_pedido):
    if aguardando_numero_pedido:
      resposta = verifica_status_pedido(input_usuario)
      aguardando_numero_pedido = False

    else:
      resposta, ids_historico_chat = responder(input_usuario, ids_historico_chat)
      if resposta == 'Could you please enter your order number? ':
          aguardando_numero_pedido = True

    historico.append((input_usuario, resposta))
    return historico, ids_historico_chat, aguardando_numero_pedido, ""

  msg.submit(
      processa_entrada,
      [msg, chatbot, estado, aguardando_numero_pedido],
      [chatbot, estado, aguardando_numero_pedido, msg]
  )

  app.launch(share=True)



/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2655b5dc3a68b63752.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
